In [17]:
#获取MNIST数据集
from sklearn.datasets import fetch_openml
mnist = fetch_openml("mnist_784")
mnist


{'data': array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]]),
 'target': array(['5', '0', '4', ..., '4', '5', '6'], dtype=object),
 'frame': None,
 'categories': {},
 'feature_names': ['pixel1',
  'pixel2',
  'pixel3',
  'pixel4',
  'pixel5',
  'pixel6',
  'pixel7',
  'pixel8',
  'pixel9',
  'pixel10',
  'pixel11',
  'pixel12',
  'pixel13',
  'pixel14',
  'pixel15',
  'pixel16',
  'pixel17',
  'pixel18',
  'pixel19',
  'pixel20',
  'pixel21',
  'pixel22',
  'pixel23',
  'pixel24',
  'pixel25',
  'pixel26',
  'pixel27',
  'pixel28',
  'pixel29',
  'pixel30',
  'pixel31',
  'pixel32',
  'pixel33',
  'pixel34',
  'pixel35',
  'pixel36',
  'pixel37',
  'pixel38',
  'pixel39',
  'pixel40',
  'pixel41',
  'pixel42',
  'pixel43',
  'pixel44',
  'pixel45',
  'pixel46',
  'pixel47',
  'pixel48',
  'pixe

In [18]:
#x实例的特征向量,y标签
x,y = mnist["data"],mnist['target']
#784个特征点=28*28像素，共70000张图
x.shape
#每张图标签
y.shape

(70000,)

In [19]:
%matplotlib osx
import matplotlib
import matplotlib.pyplot as plt

some_digit = x[36001]
#将改行重组成28X28数组
some_digit_img = some_digit.reshape(28,28)
#cmp配色方案
#图片缩放，最近邻插值
plt.imshow(some_digit_img,cmap=matplotlib.cm.binary,
           interpolation="nearest")
#关闭坐标轴，刻度，标签
plt.axis("off")
plt.show()
y[36001]

'2'

In [20]:
#MNIST数据集已经分成训练集(前6万张 图像)和测试集(最后1万张图像)
#需重新打乱
x_test = x[60000:]
y_test = y[60000:]
x_train = x[:60000]
y_train = y[:60000]
import numpy as np
#随机产生一个序列，或是返回一个排列范围
shuffle_index = np.random.permutation(60000)

x_train,y_train = x_train[shuffle_index],y_train[shuffle_index]


In [21]:
#简化问题：先查找数字2
#建立目标向量
#返回bool
y_train_2 = (y_train=='2')
y_test_2 = (y_test=='2')

In [22]:
#随机梯度下降 (SGD)分类器
from sklearn.linear_model import SGDClassifier
sgd_clf = SGDClassifier(random_state=42)
sgd_clf.fit(x_train, y_train_2)
sgd_clf.predict([some_digit])


array([ True])

In [23]:
#交叉验证
from sklearn.model_selection import cross_val_score
cross_val_score(sgd_clf,x_train,y_train_2,cv=3,scoring="accuracy")

array([0.9359, 0.9703, 0.9738])

In [24]:
#下列与sklearn自带K折交叉验证相同
from sklearn.model_selection import StratifiedKFold
from sklearn.base import clone

skfolds = StratifiedKFold(n_splits=3,random_state=42)
#在训练集中划分训练集和测试集
for train_index,test_index in skfolds.split(x_train,y_train_2):
    clone_clf = clone(sgd_clf)
    #训练集
    x_train_folds = x_train[train_index]
    y_train_2_folds = y_train_2[train_index]
    #测试集
    x_test_fold = (x_train[test_index])
    y_test_2_fold = y_train_2[test_index]
    #训练
    clone_clf.fit(x_train_folds,y_train_2_folds)
    #预测
    y_pred = clone_clf.predict(x_test_fold)
    #整合准确度
    n_correct = sum(y_pred == y_test_2_fold)
    print(n_correct / len(y_pred))


/Users/xiehong/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_split.py:293: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  warnings.warn(


0.9359
0.9703
0.9738


In [25]:
#设置分类器，将所有图设置为非2
from sklearn.base import BaseEstimator
class Never2Classifier(BaseEstimator):
    def fit(self,X,y=None):
        pass
    def predict(self,X):
        #返回X行1列全为0的矩阵，表示所有图像都判断为非
        return np.zeros((len(X),1),dtype=bool)

never2clf = Never2Classifier()
#scoring = 准确度评价标准
cross_val_score(never2clf,x_train,y_train_2,cv=3,scoring="accuracy")

array([0.904 , 0.9001, 0.898 ])

In [26]:
'''
混淆矩阵。
总体思路就是统计A类别实例 被分成为B类别的次数。
例如，要想知道分类器将数字3和数字5混淆多少次，
只需要通过混淆矩阵的第5行第3列来查看'''

from sklearn.model_selection import cross_val_predict
#每个折叠的预测。这意味 着对于每个实例都可以得到一个干净的预测(预测用的数据在训练中未出现过
#得到的是长度与实例相等的真假值的数组
y_train_pred = cross_val_predict(sgd_clf,x_train,y_train_2,cv=2)

In [27]:
'''
行表示实际类别，列表示预测类别
第一行为负类， 真负类  假正类
第二行为正类， 假负类  真正类

指标：
精度：TP/TP+FP
预测正类中正确概率
召回率:TP/TP+FN
实际正类中正确概率

F1分数：精度和召回率的谐波平均值——谐波平均值会给予较低的值更高的权重
因此只有精度和召回率都高使F1分数才会高
2/（1/精度+1/召回率）

不能同时增加精度并减少召回率，反之亦然。这称为精度/召回率权衡
'''
from sklearn.metrics import confusion_matrix
print(confusion_matrix(y_train_2,y_train_pred))
from sklearn.metrics import recall_score,precision_score
print("精度：",precision_score(y_train_2,y_train_pred))  #5074/748+5074
print("召回率:",recall_score(y_train_2,y_train_pred))     #5074/884+5074

from  sklearn.metrics import f1_score
print("F1:",f1_score(y_train_2,y_train_pred))


[[53524   518]
 [ 1382  4576]]
精度： 0.8983117393011386
召回率: 0.7680429674387378
F1: 0.8280854144046326


In [32]:
#基于决策函数计算出一个分值，如果该值大于阈值， 则将该实例判为正类，否则便将其判为负类
y_score = sgd_clf.decision_function([some_digit])
# y_score
#设置阈值
threshold = 0
y_some_digit_pred = (y_score>threshold)
y_some_digit_pred

array([False])

In [52]:
#获取训练集中所有实例的分数，但是这次需要它返回的是决策分数而不是预测结果
y_scores = cross_val_predict(sgd_clf,x_train,y_train_2,cv=3,method="decision_function")
# y_scores
#计算所有可能的阈值的精度和召回率
from sklearn.metrics import precision_recall_curve
precisions,recalls,thresholds = precision_recall_curve(y_train_2,y_scores)

In [53]:
#绘制图
def plot_precisions_recall_vs_thresholds(precisions,recalls,thresholds):
    plt.plot(thresholds,precisions[:-1],"b--",label="Precision")
    plt.plot(thresholds,recalls[:-1],"g-",label="Recall")
    plt.xlabel("Threshold")
    plt.legend()
    plt.ylim([0,1])
plt.figure(1)
plot_precisions_recall_vs_thresholds(precisions,recalls,thresholds)
plt.show()

In [54]:
#通过绘图可得精度在90的阈值
y_train_pred_90 = (y_scores > 3250)
print("precision:",precision_score(y_train_2,y_train_pred_90))
print("recall:",recall_score(y_train_2,y_train_pred_90))

precision: 0.9004683840749415
recall: 0.7744209466263847


In [55]:
#ROC曲线
#真正类率和假正类率关系曲线
from sklearn.metrics import roc_curve
#fpr:假正类率，tpr：真正类率(召回率)
fpr,tpr,thresholds = roc_curve(y_train_2,y_scores)

In [62]:
def plt_roc_curve(fpr,tpr,label=None):
    plt.plot(fpr,tpr,linewidth=2,label=label)
    plt.plot([0,1],[0,1],"k--")
    plt.axis([0,1,0,1])
    plt.xlabel("fpr")
    plt.ylabel("tpr")
plt.figure(2)
plt_roc_curve(fpr,tpr)
plt.show()

In [63]:
#测量曲线下面积(AUC)。完美的分类器的ROC AUC等于1，而纯随机分类器的ROC AUC等于0.5
from sklearn.metrics import roc_auc_score
roc_auc_score(y_train_2,y_scores)


0.9668567740488639

In [67]:
#随机森林
from sklearn.ensemble import RandomForestClassifier
forest_clf = RandomForestClassifier(random_state=42)
y_probas_forest = cross_val_predict(forest_clf,x_train,y_train_2,cv=3,method="predict_proba")
#绘制ROC曲线，需要的是分数值而不是概率大小。一个简单的解决方案是:直接使用正类的概率作为分数值
y_scores_forest = y_probas_forest[:,1]
fpr_forest,tpr_forest,thresholds_forest = roc_curve(y_train_2,y_scores_forest)
plt.figure(3)
plt.plot(fpr,tpr,"b:",label="SGD")
plt_roc_curve(fpr_forest,tpr_forest,"RandomForest")
plt.legend()
plt.show()


In [68]:
roc_auc_score(y_train_2,y_scores_forest)


0.9975742838806796

In [69]:
#自动识别多分类任务
sgd_clf.fit(x_train,y_train)
sgd_clf.predict([some_digit])

array(['2'], dtype='<U1')

In [75]:
#对应各个结果分数
some_digit_scores = sgd_clf.decision_function([some_digit])
some_digit_scores

array([[-32490.23323203, -21966.27389354,   2252.23482417,
         -2889.54336412, -23062.34164574,  -9579.88404124,
        -27453.45686067, -46457.23722959,  -4535.26086224,
        -12188.44359218]])

In [76]:
#强制一对一策略
from sklearn.multiclass import OneVsOneClassifier
ovo_clf = OneVsOneClassifier(SGDClassifier(random_state=42))
ovo_clf.fit(x_train,y_train)
ovo_clf.predict([some_digit])


array(['2'], dtype=object)

In [77]:
forest_clf.fit(x_train,y_train)
forest_clf.predict([some_digit])


array(['2'], dtype=object)

In [78]:
forest_clf.predict_proba([some_digit])


array([[0.  , 0.01, 0.98, 0.  , 0.  , 0.  , 0.  , 0.  , 0.01, 0.  ]])

In [79]:
#交叉验证
cross_val_score(sgd_clf,x_train,y_train,cv=3,scoring="accuracy")

array([0.885  , 0.87925, 0.8861 ])

In [80]:
#特征缩放以提高精度
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
x_train_scaler = scaler.fit_transform(x_train.astype(np.float64))
cross_val_score(sgd_clf,x_train_scaler,y_train,cv=3,scoring="accuracy")

/Users/xiehong/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_stochastic_gradient.py:570: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn("Maximum number of iteration reached before "


array([0.90345, 0.90235, 0.90115])

In [81]:
#通过混淆矩阵转灰度图查找错误分类高
y_train_pred = cross_val_predict(sgd_clf,x_train,y_train,cv=3)
conf_mx = confusion_matrix(y_train,y_train_pred)
#获得错误的绝对值
conf_mx

array([[5674,    1,   21,   40,    9,   39,   39,    7,   86,    7],
       [   2, 6355,   50,   46,    9,   59,   11,   16,  172,   22],
       [  49,   52, 5003,  241,   67,   46,   99,   78,  306,   17],
       [  18,   22,  105, 5365,   15,  209,   18,   70,  238,   71],
       [  23,   20,   32,   29, 5332,   29,   42,   24,  152,  159],
       [  60,   20,   24,  368,   47, 4479,   85,   18,  266,   54],
       [  58,   13,   58,   17,   35,  123, 5506,    5,  101,    2],
       [  23,   16,   85,   61,   99,   20,    5, 5661,   60,  235],
       [  40,   56,   61,  272,   30,  283,   34,   23, 4998,   54],
       [  27,   15,   21,  172,  354,  111,    3,  307,  305, 4634]])

In [83]:
#获得对应类别的数量
rows_sum = conf_mx.sum(axis=1, keepdims=True)
#对应相除获得错误率
norm_conf_mx = conf_mx/rows_sum
#0填充对角线
np.fill_diagonal(norm_conf_mx,0)
#绘制灰度图
#越亮表示错误率越高
plt.matshow(norm_conf_mx,cmap=plt.cm.gray)
plt.show()


